Orbits, with Vpython used to render the result, and ipywidgets used for interactivity.
The first cell defines the classes used, which could be reused for other demonstrations, and the second cell defines the interactions.

In [1]:
# coding: utf-8
from __future__ import division, print_function
from ipywidgets import widgets
from vpython import *
import numpy as np
import copy

class PhysicsError (Exception):
    """
    Error type defined for if two Particles get too close to simulate well
    """
    def __init__(self, exception_type):
        self.exception_type = exception_type
    def __str__(self):
        return repr(self.exception_type)

class Particle(sphere):
    """
    Class which describes a Particle under the influence of some force. Subclasses vpython sphere so drawing is no effort.
    """

    G = 1
    def __init__(self,pos = vector(0,0,0), velocity = vector(0,0,0), mass = 0.0, radius =0.0, color = color.red):
        """
        Parameters
        ----------
        pos : vpython vector
            Initial position of Particle
        velocity : vpython vector
            Initial velocity of Particle
        mass: float
            Mass of Particle (default = 0)
        radius : float
            Radius of Particle
        color: vpython color
            Color of particle
        """
        sphere.__init__(self,pos = pos, velocity = velocity, radius = radius, make_trail = True, color = color)
        self.velocity = velocity
        self.mass = mass
    def force_felt_by(self,other,if_at = None):
        '''
        Parameters
        ----------
        other: Particle
            The particle which feels the force
        if_at: vpython vector
            If this parameter is used, the function gives the force the 'other' particle would feel if it were at this position

        Subclass Particle and change this to implement custom forces, then everything else should work.
        Default(The one implemented here) is gravitational
        '''
        if  not if_at:
            if_at = other.pos
        position_difference = if_at - self.pos
        determinant = position_difference.mag
        if determinant == 0:
            return vector(0,0,0)
        velocity_determinant = self.velocity.mag
        g_force_scalar = (-1*self.G*self.mass*other.mass)/(determinant**3)
        g_force_vector = g_force_scalar * position_difference
        if determinant < self.radius + other.radius:
            raise PhysicsError("Collision ")
        return g_force_vector

    def increment_by(self,pos_increment, velocity_increment):
        '''
        Function to increment coordinates and velocity at the same time.
        Parameters
        ----------
        pos_increment: vpython vector
            Increment for pos
        velocity_increment: vpython vector
            Increment for velocity
        '''
        self.pos += pos_increment
        self.velocity += velocity_increment


class System (object):
    """Class which describes a system composed of a number of Particles."""
    planets = []

    def __init__(self,dt,G = 1):
        """
        Parameters
        ----------
        dt: float
            Specifies time increments to take
        G: float
            Gravitational constant, set to 1 by default
        """
        self.dt = dt
        self.G = G

    def set_planets(self,planets):
        """Sets the array of planets. Use this instead of planets = array_of_planets so that all the planets have the same gravitational constant"""
        for planet in planets:
            planet.G = self.G
        self.planets = planets

    def add_planets(self,planets):
        """Add array of planets to planets. Use this instead of planets.extend(array_of_planets_to_add) so that all the planets have the same gravitational constant"""
        for planet in planets:
            planet.G = self.G
        self.planets.extend(planets)

    def add_planet(self,planet):
        """Add one planet to array. Use this instead of planets.append(planet) so that all the planets have the same gravitational constant"""
        planet.G = self.G
        self.planets.append(planet)

    def move_time(self):
        """Move time forwards by one step using Euler Method"""
        old_planets = self.planets
        try:
            for counter_1,planet_1 in enumerate(self.planets):
                this_acceleration = vector(0,0,0)
                for counter_2,planet_2 in enumerate(old_planets):
                    if counter_2 != counter_1:
                        this_acceleration += planet_2.force_felt_by(planet_1)/planet_1.mass
                x_increment = planet_1.velocity*self.dt
                v_increment = this_acceleration*self.dt
                planet_1.increment_by(x_increment,v_increment)
        except PhysicsError:
            print("Collision")

    def runge_kutta_move_time(self):
        """Move time forwards by one step using RK4. Assumes gravitational field doesn't change significantly with time during one time step."""
        old_planets = self.planets
        try:
            for counter_1,planet_1 in enumerate(self.planets):
                k1 = vector(0,0,0)
                k2 = vector(0,0,0)
                k3 = vector(0,0,0)
                k4 = vector(0,0,0)
                for counter_2,planet_2 in enumerate(old_planets):
                    if counter_2 != counter_1:
                        k1 += planet_2.force_felt_by(planet_1, if_at = None)/planet_1.mass
                imagpos = planet_1.pos + (self.dt/2)*k1
                for counter_2,planet_2 in enumerate(old_planets):
                    if counter_2 != counter_1:
                        k2 += planet_2.force_felt_by(planet_1, if_at = imagpos)/planet_1.mass
                imagpos = planet_1.pos + (self.dt/2)*k2
                for counter_2,planet_2 in enumerate(old_planets):
                    if counter_2 != counter_1:
                        k3 += planet_2.force_felt_by(planet_1, if_at = imagpos)/planet_1.mass
                imagpos = planet_1.pos + (self.dt)*k3
                for counter_2,planet_2 in enumerate(old_planets):
                    if counter_2 != counter_1:
                        k3 += planet_2.force_felt_by(planet_1, if_at = imagpos)/planet_1.mass
                x_increment = planet_1.velocity*self.dt
                v_increment = (self.dt/6)*(k1 + 2*k2 + 2*k3 + k4)
                self.planets[counter_1].increment_by(x_increment,v_increment)
        except PhysicsError:
            print("Collision")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
def b_handler(s):
    global running
    running = True
    if running:
        dwarf_planet.pos = vector(x.value,0,0)
        dwarf_planet.velocity = vector(0,0,velocity.value)
        dwarf_planet.make_trail = False
        dwarf_planet.make_trail = True
        print(velocity)
        while True:
            rate(30)
            system.runge_kutta_move_time() 


scene1 = canvas(title = "Orbits")
scene1.forward = vector(0,1,0)
scene1.caption = """Right button drag or Ctrl-drag to rotate "camera" to view scene.
To zoom, drag with middle button or Alt/Option depressed, or use scroll wheel.
  On a two-button mouse, middle is left + right.
Touch screen: pinch/extend to zoom, swipe or two-finger rotate."""
giant_planet = Particle(pos = vector(0,0,0), velocity = vector(0, 0, 0), mass = 200000, radius = 20)
dwarf_planet = Particle(pos = vector(200,0,0), velocity = vector(0, 0, 31.622), mass = 1, radius = 10)

dt = 0.1
system = System(dt)
planets_array = [giant_planet, dwarf_planet]
system.planets = planets_array

running = False

b = widgets.Button(description='Update')
display(b)
b.on_click(b_handler)

x = widgets.FloatSlider(description='Radius:', min=100, max=500, step=1, value=200)
display(x)
velocity = widgets.FloatSlider(description='Velocity:', min=20, max=100, step=1, value=31)
display(velocity)



<IPython.core.display.Javascript object>